# Building Recommender System with Spotify Data

### Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spotipy
import spotipy.util as util
import os
import sys
import yaml
import json
import argparse
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from plotly.subplots import make_subplots
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from spotipy.oauth2 import SpotifyClientCredentials
from collections import defaultdict
from collections import defaultdict
from scipy.spatial.distance import cdist
from pprint import pprint
from yellowbrick.cluster import KElbowVisualizer
from kneed import KneeLocator
sns.set()

## Mainpulating Spotify Dataset

### Reading the Data

In [ ]:
spotify_data = pd.read_csv('./data/us_charts_with_audio_features.csv')
spotify_data.head(5)

In [ ]:
spotify_data.info()

### Principal Component Analysis (PCA)

In [ ]:
# Remove rows with invalid values
spotify_data.dropna(inplace=True)

# Select a subset of columns to use in the clustering process
columns_to_use = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']
X = spotify_data[columns_to_use]

scaler = StandardScaler()
X_std = scaler.fit_transform(X)

In [ ]:
pca = PCA()
pca.fit(X_std);

In [ ]:
evr = pca.explained_variance_ratio_
evr

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.plot(range(1, len(X.columns)+1), evr.cumsum(), marker='o', linestyle='--')
plt.xlabel('Number of Components', fontsize=18)
plt.ylabel('Cumulative Explained Variance',fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.show()

In [ ]:
for i, exp_var in enumerate(evr.cumsum()):
    if exp_var >= 0.8:
        n_comps = i + 1
        break
print("Number of components:", n_comps)
pca = PCA(n_components=n_comps)
pca.fit(X_std)
scores_pca = pca.transform(X_std)

### K-Means Clustering

In [ ]:
# Finding the elbow point of the WCSS (within cluster sum of squares) curve using the YellowBrick KElbowVisualizer
visualizer = KElbowVisualizer(KMeans(init='k-means++', random_state=42), k=(1,21), timings=False)
visualizer.fit(scores_pca)
visualizer.show()
n_clusters = visualizer.elbow_value_
print("Optimal number of clusters:", n_clusters)

In [ ]:
# Finding the elbow point of the WCSS (within cluster sum of squares) curve using the kneed KneeLocator
wcss = []
max_clusters = 21
for i in range(1, max_clusters):
    kmeans_pca = KMeans(i, init='k-means++', random_state=42)
    kmeans_pca.fit(scores_pca)
    wcss.append(kmeans_pca.inertia_)
n_clusters = KneeLocator([i for i in range(1, max_clusters)], wcss, curve='convex', direction='decreasing').knee
print("Optimal number of clusters", n_clusters)

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.plot(range(1, 21), wcss, marker='o', linestyle='--')
plt.vlines(KneeLocator([i for i in range(1, max_clusters)], wcss, curve='convex', direction='decreasing').knee, ymin=min(wcss), ymax=max(wcss), linestyles='dashed')
plt.xlabel('Number of Clusters', fontsize=18)
plt.ylabel('Within Cluster Sum of Squares (WCSS)', fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.show()

In [ ]:
kmeans_pca = KMeans(n_clusters=n_clusters, init='k-means++', random_state=42)
kmeans_pca.fit(scores_pca);

### Analysis and Visualization

In [ ]:
df_seg_pca_kmeans = pd.concat([X.reset_index(drop=True), pd.DataFrame(scores_pca)], axis=1)
df_seg_pca_kmeans.columns.values[(-1*n_comps):] = ["Component " + str(i+1) for i in range(n_comps)]
df_seg_pca_kmeans['Cluster'] = kmeans_pca.labels_
df_seg_pca_kmeans.head()

In [ ]:
# x = df_seg_pca_kmeans['Component 2']
# y = df_seg_pca_kmeans['Component 1']
fig = plt.figure(figsize=(10, 8))
sns.scatterplot(data=df_seg_pca_kmeans, x='Component 2', y='Component 1', hue=df_seg_pca_kmeans['Cluster'], palette = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'goldenrod', 'tab:cyan'])
plt.title('Clusters by PCA Components', fontsize=20)
plt.xlabel("Component 2", fontsize=18)
plt.ylabel("Component 1", fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.show()

In [ ]:
spotify_data['cluster_label'] = df_seg_pca_kmeans['Cluster']
spotify_data.head(5)

### Visualize Song Clusters with PCA

In [ ]:
# PCA for dimension reduction (faster than t-SNE)
pca_pipeline = Pipeline([('scaler', StandardScaler()), ('PCA', PCA(n_components=2))])
song_embedding = pca_pipeline.fit_transform(X)

projection = pd.DataFrame(columns=['x', 'y'], data=song_embedding)
projection['title'] = spotify_data['name']
projection['cluster'] = spotify_data['cluster_label']

In [ ]:
# Visualize song clusters in a 2D space
fig = px.scatter(
    projection, x='x', y='y', color='cluster', hover_data=['x', 'y', 'title'])
fig.show()

## Building Content-Based Recommender System

### Utility Functions

In [ ]:
# Establish spotipy connection
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=os.environ["SPOTIFY_CLIENT_ID"],
                                                           client_secret=os.environ["SPOTIFY_CLIENT_SECRET"]))

# Returns a dataframe with data for a song given the name and artist.
# Uses Spotipy to fetch audio features and metadata for the specified song.
def find_song(name, artist):
    song_data = defaultdict()
    results = sp.search(q= 'track: {} artist: {}'.format(name, artist), type='track', limit=1)
    if results['tracks']['items'] == []:
        return None
    
    results = results['tracks']['items'][0]

    track_id = results['id']
    audio_features = sp.audio_features(track_id)[0]
    
    song_data['name'] = [name]
    song_data['artists'] = [artist]
    
    for key, value in audio_features.items():
        song_data[key] = value
    
    return pd.DataFrame(song_data)

In [ ]:
# Gets the song data for a specific song
# Song argument is a dictionary with key-value pairs for the name and artist
def get_song_data(song, spotify_data):
    # Check if song is in the spotify dataset, otherwise use find_song method
    try:
        song_data = spotify_data[(spotify_data['name'] == song['name']) 
                                & (spotify_data['artists'] == song['artist'])].iloc[0]
        return song_data
    except IndexError:
        return find_song(song['name'], song['artist'])

In [ ]:
# Calculates mean vector from a list of songs based on audio/metadata features
def get_mean_vector(song_list, spotify_data):
    song_vectors = []
    
    # Add all songs to song_vectors
    for song in song_list:
        song_data = get_song_data(song, spotify_data)
        if song_data is None:
            print('Warning: {} does not exist in Spotify or in database'.format(song['name']))
            continue
        song_vector = song_data[columns_to_use].values
        song_vectors.append(song_vector)
    
    # Convert to numpy array then use np.mean
    song_matrix = np.array(list(song_vectors))
    return np.mean(song_matrix, axis=0)

In [ ]:
# Flattens a list of dictionaries.
def flatten_dict_list(dict_list):
    flattened_dict = defaultdict()
    for key in dict_list[0].keys():
        flattened_dict[key] = []
    
    for dictionary in dict_list:
        for key, value in dictionary.items():
            flattened_dict[key].append(value)
            
    return flattened_dict

### Recommender Function

In [ ]:
# Recommends songs based on a list of previous songs that a user has listened to.
def recommend_songs(song_list, spotify_data, n_songs=10):
    # Compute average vector of input songs
    song_center = get_mean_vector(song_list, spotify_data)

    # Scale the data
    scaled_data = scaler.transform(spotify_data[columns_to_use])
    scaled_song_center = scaler.transform(song_center.reshape(1, -1))

    # Find closest songs in dataset to the average vector using cosine distance
    distances = cdist(scaled_song_center, scaled_data, 'cosine')
    index = list(np.argsort(distances)[:, :n_songs][0])
    
    # Recommend corresponding songs from the dataset
    rec_songs = spotify_data.iloc[index]
    song_dict = flatten_dict_list(song_list)
    rec_songs = rec_songs[~rec_songs['name'].isin(song_dict['name'])]

    # Format output
    metadata_cols = ['name', 'artists', 'url']
    return rec_songs[metadata_cols].to_dict(orient='records')

## Testing Recommender

In [ ]:
recommend_songs([{'name': 'HiTek Tek', 'artist': 'Future'},
                {'name': 'Ridin Strikers', 'artist': 'Future'},
                {'name': 'Touch The Sky', 'artist': 'Future'},
                {'name': 'One Of My', 'artist': 'Future'},
                {'name': 'Hard To Choose One', 'artist': 'Future'},
                {'name': 'Solitaires (feat. Travis Scott)', 'artist': 'Future'},
                {'name': 'Harlem Shake (feat. Young Thug)', 'artist': 'Future'},
                {'name': 'Too Comfortable', 'artist': 'Future'}], spotify_data, 10)

In [ ]:
recommend_songs([{'name': 'SAD!', 'artist': 'XXXTentacion'},
                {'name': 'Lucid Dreams', 'artist': 'Juice WRLD'},
                {'name': 'All Girls Are The Same', 'artist': 'Juice WRLD'},
                {'name': 'Jocelyn Flores', 'artist': 'XXXTentacion'},
                {'name': 'Fuck Love (feat. Trippie Redd)', 'artist': 'XXXTentacion'},
                {'name': 'Hope', 'artist': 'XXXTentacion'}], spotify_data, 10)